In [ ]:
pip install rank_bm25

In [ ]:
pip install pyaspeller

In [ ]:
pip install transliterate

In [ ]:
pip install pymorphy2

In [1]:
import pandas as pd
import os
import numpy as np
import json
from sklearn.metrics import ndcg_score
import pyarrow.parquet as pq
import time
import os
import pickle
import polars as pl
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy.sparse import hstack
import cudf
#from cuml.feature_extraction.text import TfidfVectorizer,CountVectorizer
from cuml.neighbors import NearestNeighbors
from catboost import CatBoostRanker, Pool
from functools import partial
import re
#from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MaxAbsScaler
from pyaspeller import YandexSpeller
from sklearn.decomposition import TruncatedSVD, PCA
from transliterate import translit
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score, top_k_accuracy_score

In [2]:
automarkup = pd.read_parquet('input_/automarkup.parquet',engine = 'fastparquet')
sample_submission = pd.read_csv('input_/sample_submission.csv')
manualmarkup = pd.read_csv('input_/manualmarkup.csv')
videos = pd.read_parquet('input_/videos.parquet', engine = 'fastparquet')

In [3]:
def calculate_metric(test_auto, submission, DEBUG = False):
    
    test_auto['target'] = [1] * test_auto.shape[0]
    test_auto = test_auto[['query', 'video_id', 'target']].drop_duplicates()

    with_target = submission.merge(
        test_auto,
        how = 'left',
        on = ['query', 'video_id'],
    )

    with_target['target'] = with_target['target'].fillna(0.0)

    y_true = with_target.groupby('query').agg(list)['target'].values.tolist()
    
    y_score = np.array([[5.0, 4.0, 3.0, 2.0, 1.0] for _ in range(len(y_true))])
   
    score = 0
    for i in range(len(y_true)):
        try:
            true = np.array(y_true[i]).astype(int)[:5]
            val = y_score[i].astype(int)[:5]            
            score += ndcg_score([true], [val])
        except:
            if DEBUG:
                print(true)
                print(val)
            
    score /= len(y_true)     
    
    return score

In [4]:
class RutubeSearchModel:
    def __init__(self, top_k: int) -> None:
        self._vec = TfidfVectorizer(analyzer='word', ngram_range=(1, 1),max_df=0.9)
       # self._vec = CountVectorizer(analyzer= 'char_wb', ngram_range=(1, 1),max_df=0.9)
        self._knn = NearestNeighbors(metric = 'cosine')
        self.speller = YandexSpeller()
        self._top_k = top_k
        
    def fit(self, videos: pd.DataFrame):
        videos['search_tag'] = videos['video_title'].astype(str) + " " + videos['channel_title'].astype(str) + " " + videos['v_category'].astype(str)
        videos = self.text_preprocessing(videos)
        self._videos = videos[['video_id','video_title']]
       
        self._mx_videos = self._vec.fit_transform(videos['search_tag'])
        self._knn.fit(self._mx_videos)
        return self

    def predict_front(self, test):
        query = test
        test = self.normalize_string(test)
        test = self.speller.spelled(test)
       # pars = self.parse_russian_words(test)
       # if pars != test:
        #    test = pars
            
      #  if self.has_russian_words(test) == False:
           #  test = self.find_foreign_layout_characters(test) 
                
      #  test = self.speller.spelled(test)
        print(test)
        try:
                        
            mx_predict = self._vec.transform([test])
            distances, indices = self._knn.kneighbors(mx_predict, n_neighbors = self._top_k+20)
            search = self._videos.iloc[indices.flatten()]
            search['neighbors'] = distances.flatten()
            search['video_title'] = search['video_title'].str.lower()
            corpus = search['video_title'].values.tolist()
            tokenized_corpus = [doc.split(" ") for doc in corpus]
            bm25 = BM25Okapi(tokenized_corpus)
            text_speller = self.speller.spelled(test)
            tokenized_query = text_speller.lower().split(" ")
            search['bm25_score'] = bm25.get_scores(tokenized_query)
            
        except:
            search = pd.DataFrame(columns=['query', 'video_id','bm25_score'],index = range(self._top_k))
            search['query'] = query
            search['bm25_score'] = 0
            search['video_id'] = 'video_9572889'
            
        return search.sort_values(by='bm25_score', ascending = False).head(self._top_k)
    
    def predict_submission(self, test):
        
        sub = []
        for i in tqdm(range(len(test))):
            search = self.predict_front(test['query'].iloc[i])['video_id'].tolist()
            for j in search:
                sub.append([test['query'].iloc[i],j])
        return pd.DataFrame(sub, columns = ['query', 'video_id'])
    
    def text_preprocessing(self,videos):
        videos['search_tag'] = videos['search_tag'].str.lower()
        return videos
    
    def parse_russian_words(self, text):
        cyrillic = translit(text, 'ru')
        if cyrillic != text:
            russian_text = cyrillic
        else: 
            russian_text = text
        # russian_text = translit(english_text, 'ru')
        return russian_text
    
    def normalize_string(self, s):
      
        s = s.lower()
        s = re.sub(r'([\D]*)([0-9]+)', r' \1 \2 ', s)
        return s
    
    def find_foreign_layout_characters(self,text):
        foreign_layout_characters = {'q': 'й','w': 'ц','e': 'у','r': 'к', 't': 'е',
                                     'y': 'н','u': 'г','i': 'ш','o': 'щ', 'p': 'з',
                                     '[': 'х',']': 'ъ','a': 'ф','s': 'ы', 'd': 'в',
                                     'f': 'а','g': 'п','h': 'р','j': 'о','k': 'л',
                                     'l': 'д',';': 'ж',"'": 'э','z': 'я','x': 'ч', 
                                     'c': 'с','v': 'м','b': 'и','n': 'т','m': 'ь',
                                     ',': 'б','.': 'ю','/': '.' }

        foreign_layout_chars = []
        for char in text:
            if char == ' ':
                foreign_layout_chars.append(char)
            else:
                if char.lower() in foreign_layout_characters.keys():
                    foreign_layout_chars.append(foreign_layout_characters[char.lower()])

        return ''.join(map(str, foreign_layout_chars))
    
    def has_russian_words(self, text):
        russian_words_regex = re.compile('[а-яА-ЯёЁ]')
        return bool(russian_words_regex.search(text))   

In [5]:
model_search = RutubeSearchModel(top_k=5)

In [6]:
%%time
model_search.fit(videos)

CPU times: user 3min 47s, sys: 21 s, total: 4min 8s
Wall time: 4min 8s


In [7]:
%%time
model_search.predict_front('моя фамилия шилов').head(5)

моя фамилия шилов
CPU times: user 827 ms, sys: 81.3 ms, total: 908 ms
Wall time: 1.65 s


,video_id,video_title,neighbors,bm25_score
10901411,video_11799605,моя фамилия шилов,0.205449,2.683917
3994165,video_11127290,моя фамилия шилов (2013),0.205315,2.388476
1680037,video_34015596,моя фамилия шилов (2013),0.284632,2.388476
13399936,video_16958870,моя фамилия шилов (2013) фильм,0.243140,2.151629
24535415,video_14203175,фамилия моя,0.522629,2.120455


In [8]:
%%time
model_search.predict_front('vtynjdcrbt djqys 7 ctpjy').head(5)

  ментовские войны  7  сезон
CPU times: user 660 ms, sys: 40 ms, total: 700 ms
Wall time: 1.06 s


,video_id,video_title,neighbors,bm25_score
10403934,video_4675942,ментовские будни,0.411779,14.700951
16558182,video_9572889,ментовские войны 7 сезон 20 серия,0.406426,4.826360
28358877,video_7255498,ментовские войны 1 сезон 06 серия,0.300943,2.166161
15966583,video_26867692,ментовские войны 3 сезон 5 серия,0.226747,2.166161
5875851,video_26363202,ментовские войны 1 сезон 9 серия,0.218117,2.166161


In [9]:
%%time
model_search.predict_front('санкт петербург').head(5)

санкт петербург
CPU times: user 655 ms, sys: 24 ms, total: 679 ms
Wall time: 1.02 s


,video_id,video_title,neighbors,bm25_score
4398753,video_6213816,санкт петербург ❤️,0.183011,5.124438
3169412,video_14933229,санкт - петербург,0.239210,5.124438
25176638,video_485956,санкт-петербург 🇷🇺 2023,0.046067,0.000000
538553,video_17013850,россия с.петербург 3* санкт-петербург обзор – ...,0.233696,0.000000
16031789,video_8054985,бизнес-петербург / 25 апреля,0.249649,0.000000


In [10]:
#Проверим BM25 на ручной разметке "exact"
val_df = manualmarkup[manualmarkup['sentiment']=="exact"][['query','video_id']]
val_df = val_df.reset_index(drop=True)

In [11]:
sub_df = model_search.predict_submission(val_df)

  0%|                                                   | 0/404 [00:00<?, ?it/s]

 экстрасенсы битва сильнейших  23  сезон


  0%|                                           | 1/404 [00:01<07:50,  1.17s/it]

богато жить не запретишь 


  0%|▏                                          | 2/404 [00:02<07:17,  1.09s/it]

...

100%|████████████████████████████████████████▊| 402/404 [07:19<00:02,  1.04s/it]

 лесник своя земля  23  серия


100%|████████████████████████████████████████▉| 403/404 [07:20<00:01,  1.05s/it]

остров джеймса бонда | тайланд | пхукет | пхи-пхи


100%|█████████████████████████████████████████| 404/404 [07:21<00:00,  1.09s/it]


In [12]:
calculate_metric(val_df, sub_df)

0.14421183916326497

## Submission

In [8]:
test_data = pd.read_csv('test_data/test_dataset_submission_queries.csv')

In [11]:
submission = model_search.predict_submission(test_data)

  0%|                                                  | 0/2000 [00:00<?, ?it/s]

 битва сильнейших экстрасенсов  2023   смотреть | экстрасенсы битва сильнейших  35  ,  36  серия


  0%|                                          | 1/2000 [00:01<40:08,  1.21s/it]

  битва сильнейших экстрасенсов  2023  \n битва сильнейших экстрасенсов  2023  смотреть


  0%|                                          | 2/2000 [00:02<40:16,  1.21s/it]

...

100%|██████████████████████████████████████▉| 1998/2000 [35:56<00:02,  1.09s/it]

шоу вована и лексуса


100%|██████████████████████████████████████▉| 1999/2000 [35:57<00:01,  1.08s/it]

  мужское женское выпуск от  04    06    2021 


100%|███████████████████████████████████████| 2000/2000 [35:59<00:00,  1.08s/it]


In [12]:
len(set(submission['query']) - set(test_data['query']))

0

In [13]:
submission.to_csv('submission_bm25.csv',index=False)

In [15]:
submission.nunique()

query       2000
video_id    4926
dtype: int64

In [18]:
submission['video_id'].value_counts()

video_27726632    104
video_29650298     98
video_30146879     72
video_17011910     56
video_6195873      50
                 ... 
video_23771507      1
video_1517546       1
video_16533962      1
video_9106192       1
video_4911051       1
Name: video_id, Length: 4926, dtype: int64